# Project: Electric Vehicle Charging Scheduling Problem (EVCS)
## First deliverable
## Prescriptive Analytics: Heuristics for Decision Making
### Wilmar Calderón - 201630701

### **Mathematical Formulation of the Electric Vehicle Charging Scheduling (EVCS) Problem**

The **EV Charging Scheduling -EVCS-** can be mathematically formulated as follows:

#### **Decision Variables**  


#### **Objective Function**


#### **Parameters**


#### **Constraints**



In [34]:
import json
import pandas as pd
from typing import Dict, Tuple
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [35]:
def load_json_to_dfs(file_path: str) -> Tuple[Dict[str, pd.DataFrame], int, int]:
    """
    Carga un archivo JSON y lo convierte en múltiples DataFrames de pandas.
    También extrae el número de puestos de parqueo (n_spot) y el límite del transformador.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    dfs = {}

# Extraer información del parking_config
    parking_config = data.get('parking_config', {})
    n_spot = parking_config['n_spots']
    transformer_limit =parking_config['transformer_limit'] 

    dfs['energy_prices'] = pd.DataFrame(data['energy_prices'], columns=['time', 'price'])
    dfs['arrivals'] = pd.DataFrame(data['arrivals'], columns=['id', 'arrival_time', 'departure_time', 'required_energy'])
    dfs['chargers'] = pd.DataFrame(parking_config['chargers'],columns=["charger_id","power"])
    
    return dfs, n_spot, transformer_limit



In [36]:
# Ejemplo de uso
file_path = 'test_system_1.json'
dfs, n_spot, transformer_limit = load_json_to_dfs(file_path)
energy_p=dfs["energy_prices"]
arrivals=dfs["arrivals"]
chargers=dfs["chargers"]
print(f"\nFile: {file_path}")
print(f"Number of spots: {n_spot}")
print(f"Transformer limit: {transformer_limit}")



File: test_system_1.json
Number of spots: 10
Transformer limit: 35


In [37]:
#Parameters
delta_max=0.17 #Maximum allowed delay
step=0.1 #spacing for the time 
lambda_c=max(energy_p["price"])*10 #penalty
N=arrivals["id"].tolist()

M=chargers.shape[0] #M=Number of chargers
T_max=max(arrivals["departure_time"]) #Detail of the last departure
T=list(np.round(np.arange(0,T_max+step,step),2)) #List of the time periods analyzed

charger_power = chargers.set_index('charger_id')['power'].to_dict()

C = chargers['charger_id'].tolist()
t_a = dict(zip(arrivals['id'], arrivals['arrival_time']))
t_d = dict(zip(arrivals['id'], arrivals['departure_time']))
e_req=dict(zip(arrivals['id'], arrivals['required_energy']))
print(charger_power)
print(T)
print(M)
print(T_max)
print(lambda_c)

energy_price_dict = dict(zip(energy_p['time'], energy_p['price']))
energy_prices_df = pd.DataFrame(list(energy_price_dict.items()), columns=["time", "price"]).sort_values("time")
T_df = pd.DataFrame({"time": list(T)}).sort_values("time")
T_df = pd.merge_asof(T_df, energy_prices_df, on="time", direction="backward")
e_t = dict(zip(T_df["time"].round(2), T_df["price"]))

print(energy_price_dict)
print(e_t)


{0: 7, 1: 7, 2: 7, 3: 7, 4: 7}
[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13.0, 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14.0]
5
14.0
468.9429231677648
{0.0: 21.484523228602225, 0.25: 20.92212173473019, 0.5: 23.74278593825559, 0.75: 26.573193006197716, 1.0: 32.732944063397795, 1.25: 35.49020314715779, 1.5: 41.31172193669754, 1.75: 46.8239188687373

In [38]:
from gurobipy import Model, GRB, quicksum

model = gp.Model("ev_charging")
# Variables
P = model.addVars(N, T, name="P", lb=0.0)
xi = model.addVars(N, C, T, vtype=GRB.BINARY, name="xi")
s = model.addVars(N, name="s", lb=0.0)  # Slack variable

# Función objetivo
model.setObjective(
    quicksum(P[n, t] * e_t[t] for n in N for t in T)+ quicksum(1000 * s[n] for n in N),
    GRB.MINIMIZE
)

#Restriction 1
# Limit of the chargers
for c in C:
    for t in T:
        model.addConstr(
            quicksum(xi[n, c, t] for n in N) <= M,
            name=f"charger_usage_limited_{c}_{t}"
        )

#Restriction 2
# Charging station transformer limit
for t in T:
    model.addConstr(
        quicksum(P[n, t] for n in N) <= transformer_limit *step,
        name=f"transformer_limit_{t}"
    )

#Restriction 3
# Charger Capacity and activation
for n in N:
    for c in C:
        for t in T:
            model.addConstr(
                P[n, t] <= xi[n, c, t] * charger_power[c] *step,
                name=f"charger_power_limit_{n}_{c}_{t}"
            )


#Restriction 4
# Charging of the EV must be larger or equal to the requirement of the EV. This will consider that the variable s_n is activated as a slack variable to model that all EV's have been reviewed.
for n in N:
    model.addConstr(
        quicksum(P[n, t] for t in T if t_a[n] <= t <= t_d[n])+ s[n] >= e_req[n],
        name=f"energy_requirement_{n}"
    )

# Restriction 5
# Charging can only happen between the arrival and departure time.
for t in T:
    for n in N:
        if not (t_a[n] <= t <= t_d[n]):
            model.addConstr(P[n, t] == 0, name=f"outside_time_{n}_{t}")



In [39]:
# Optimization

# model.setParam(GRB.Param.MIPGap, 0.0001) #Uncomment if you want to modify the gap. Lower gaps will require more processing
model.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26929 rows, 27104 columns and 76736 nonzeros
Model fingerprint: 0x90349e1c
Variable types: 4544 continuous, 22560 integer (22560 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 3e+01]
Found heuristic solution: objective 537604.85893
Found heuristic solution: objective 537604.85885
Presolve removed 24030 rows and 23646 columns
Presolve time: 0.08s
Presolved: 2899 rows, 3458 columns, 9154 nonzeros
Found heuristic solution: objective 524057.15097
Variable types: 1003 continuous, 2455 integer (2455 binary)

Root relaxation: objective 1.114818e+05, 2733 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Curre

In [ ]:
# Mostrar resultados
if model.status == GRB.OPTIMAL:
    kpi1=0 #Provided Total Energy
    kpi2=0 # Total Charging Cost
    kpi_l=[] #List for the dissatisfied users to calculate KPI 3 and 4
    admissible=0.7

    for n in N:
        # print(f"EV: {n}:")

        e_req_n=e_req[n]
        aux=0 #auxilliary variable to get: sum(P_{n,t} if P_{n,t}>0)
        for t in T:
            if P[n, t].X > 0:
                kpi1+=P[n, t].X
                aux+=P[n, t].X
                kpi2+=P[n, t].X*e_t[t]
                # print(f"  Time {t}: Charge {P[n, t].X:.2f} kWh")
        # print(f"Provided total Energy (kWh): {aux} kWh")
        if aux<e_req_n*admissible:
            kpi_l.append((e_req_n-aux)/e_req_n)

    #Stores the charging sequence in an Excel File
    results = pd.DataFrame(index=T, columns=N)
    for n in N:
        for t in T:
            results.loc[t, n] = P[n, t].X if P[n, t].X > 0 else 0
    export_file="ev_charging_results.xlsx"
    results.to_excel(export_file)



    kpi3=len(kpi_l)
    print("Results stored in: "+export_file)

    # print(f"Value of the Objective Function: {model.objVal:.2f}")
    print(f"Requested total Energy: {sum(e_req):.2f} kWh")    
    print(f"Provided total Energy: {kpi1:.2f} kWh")
    print(f"Scheduling performance: {kpi1/sum(e_req)*100:.2f} %")
    print(f"Total Charging Cost: {kpi2:.2f} ")
    print(f"Dissatisfaction level threshold: {admissible:2.0%}")
    print(f"Number of Dissatisfied users: {kpi3}")
    


Results stored in: ev_charging_results.xlsx
Value of the Objective Function: 111505.76
Requested total Energy: 496.00 kWh
Provided total Energy: 437.37 kWh
Scheduling performance: 88.18 %
Total Charging Cost: 11269.19 
Dissatisfaction level threshold: 70%
Number of Dissatisfied users: 7
